![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `mistralai/mistral-large` to make simple chat conversation and tool calls

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use Chat models, e.g. `mistralai/mistral-large` by using tools.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install -U "ibm_watsonx_ai>=1.1.14" | tail -n 1

### Define the WML credentials
Use the code cell below to define the WML credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your WML API key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [2]:
model_id = "mistralai/mistral-large"

### Define the model parameters

You might need to adjust model `parameters` depending on the model you use.

In [3]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-------------------+-----------------------------------------------------------------------------------+-------------------------+
| PARAMETER         | TYPE                                                                              | EXAMPLE VALUE           |
+===================+===================================================================================+=========================+
| frequency_penalty | float | None                                                                      | 0.5                     |
+-------------------+-----------------------------------------------------------------------------------+-------------------------+
| logprobs          | bool | None                                                                       | True                    |
+-------------------+-----------------------------------------------------------------------------------+-------------------------+
| top_logprobs      | int | None                                            

In [4]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [5]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

#### Work with a simple chat message

In [6]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(messages=messages)

In [7]:
print(simple_chat_response["choices"][0]["message"]["content"])

 The sum of 1 + 1 is 2. Here it is:

 1
+1
----
 2


#### Work with a simple chat message using chat_stream

In [16]:
messages = [
    {
        "role": "user", 
        "content": "What IBM mainly does?"
    }
]

simple_chat_stream_response = model.chat_stream(messages=messages)

In [14]:
for chunk in simple_chat_stream_response:
    print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

 IBM (International Business Machines Corporation) is a multinational technology and consulting company that operates in more than 170 countries. Here are some of the main areas that IBM is involved in:

1. **Technology and Consulting Services**: IBM Global Services is the world's largest business and technology services provider. It offers consulting, software development, and IT services.

2. **Cloud Computing**: IBM Cloud offers a wide range of services for public, private, and hybrid cloud environments. It includes Infrastructure as a Service (IaaS), Software as a Service (SaaS), and Platform as a Service (PaaS) offerings.

3. **Artificial Intelligence (AI) / Cognitive Computing**: IBM Watson is a flagship product in this area, offering a range of AI services, including natural language processing, machine learning, and predictive analytics.

4. **Software**: IBM develops, manufactures, and licenses software for various domains. This includes operating systems (like z/OS), middlewa

#### Work with an advanced chat message

In [18]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(messages=messages)

In [19]:
print(advanced_chat_response["choices"][0]["message"]["content"])

 The 2020 World Series was played at Globe Life Field in Arlington, Texas. Due to the COVID-19 pandemic, the entire series was played at a neutral site, which was the new home of the Texas Rangers. It was the first time since 1944 that the World Series was played at a single location. The Dodgers defeated the Tampa Bay Rays in six games to win their seventh World Series title.


#### Work with chat messages using `tools` and `tool_choice`

In [20]:
messages = [
    {"role": "user", "content": "What's the weather like in Boston today?"},
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        },
    }
]

tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}

tool_response = model.chat(messages=messages, tools=tools, tool_choice=tool_choice)

In [21]:
import json

print(json.dumps(tool_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "content": "",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-32a4c362dc0947828273256824b2fd5d",
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "arguments": "{\"location\":\", 'Boston');\"}"
            }
        }
    ]
}


#### Work with chat messages using `tools` and `tool_choice_option`

In [22]:
tools = [
    {
        'function': {
            'description': 'Adds a and b.',
            'name': 'add',
            'parameters': {
                'properties': {
                    'a': {'type': 'float'},
                    'b': {'type': 'float'}},
                'required': ['a', 'b'],
                'type': 'object'}
        },
        'type': 'function'
    },
    {
        'function': {
            'description': 'Multiplies a and b.',
            'name': 'multiply',
            'parameters': {
                'properties': {
                    'a': {'type': 'float'},
                    'b': {'type': 'float'}},
                'required': ['a', 'b'],
                'type': 'object'}
        },
        'type': 'function'
    }
]

tool_choice_option = "auto"

In [23]:
messages = [
    {"role": "user", "content": "What is 5 + 6?"},
]

tools_response = model.chat(messages=messages, tools=tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-5c3ee32a0b5247a98212f551d10a7fd1",
            "type": "function",
            "function": {
                "name": "add",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


In [24]:
messages = [
    {"role": "user", "content": "What is 5 * 6?"},
]

tools_response_2 = model.chat(messages=messages, tools=tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response_2["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-55be344ed51b4dbdb4a1a4d19199b650",
            "type": "function",
            "function": {
                "name": "multiply",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.